In [1]:
# Dependencies
import time
import pandas as pd
import requests
import json
import numpy as np
from datetime import date, datetime

from sqlalchemy import create_engine

import pymysql
pymysql.install_as_MySQLdb()

# Config Variables 
from config import remote_db_endpoint, remote_db_port
from config import remote_gwsis_dbname, remote_gwsis_dbuser, remote_gwsis_dbpwd
from config import local_db_user, local_db_pwd, local_db_endpoint, local_db_port, local_db_name



    # Dataframe to SQL
    

    # Set timer to run every 24 hours...timer is in seconds

# # counter just to count how many days it's been running
# counter = 0

# # Infinite loop
# while(True):

#     # Call the TweetQuotes function and specify the tweet number
#     currseasonscorersscrape()

#     # Once tweeted, wait 24 hours before doing anything else
#     time.sleep(86400)

#     # Add 1 to the counter prior to re-running the loop
#     counter += 1


## SQLAlchemy ##

# Create engine and pass mySQL connnection 
# current_scorers_df = df

# Cloud MySQl database connection on AWS
# engine = create_engine(f"mysql://{remote_gwsis_dbuser}:{remote_gwsis_dbpwd}@{remote_db_endpoint}:{remote_db_port}/{remote_gwsis_dbname}")

# conn = engine.connect()

# Dataframe to SQL
# current_scorers_df.to_sql(name='current_scorers_data', if_exists='replace', con=conn, chunksize=500, index=False)

# Check for Accuracy
# remote_current_scorers_data = pd.read_sql("SELECT * FROM current_scorers_data", conn)
# print(len(remote_current_scorers_data))
# remote_current_scorers_data

# def drop_table(current_scorers_data):
#    engine = create_engine(URL(**DATABASE))
#    base = declarative_base()
#    metadata = MetaData(engine, reflect=True)
#    table = metadata.tables.get(table_name)
#    if table is not None:
#        logging.info(f'Deleting {table_name} table')
#        base.metadata.drop_all(engine, [table], checkfirst=True)

# drop_table('users')

In [2]:

# Cloud MySQl database connection on AWS
engine = create_engine(f"mysql://{remote_gwsis_dbuser}:{remote_gwsis_dbpwd}@{remote_db_endpoint}:{remote_db_port}/{remote_gwsis_dbname}")

#Get top 20 active scorers from hockey reference:
currseason = 2020
seasonname = f'{currseason - 1}-{currseason}'
url= f'https://www.hockey-reference.com/leaders/goals_active.html'
currscorers = pd.read_html(url,header=1)
time.sleep(5)
currscorers = currscorers[0]
currscorers = currscorers.reset_index().T.reset_index().T
currscorers.reset_index(inplace=True, drop=True)
currscorers = currscorers[2].head(20)

currscorerslist = []

for scorers in currscorers:
    currscorerslist.append(scorers)

#Read in PlayerDF database.  This will need to be replaced with reading the playerdf database table into a DF.  This code creates a new column in the playerdf database, so we can look up to it to get player ids.
playerdf = pd.read_sql('SELECT * FROM player_data', con=engine)
playerdf["FL"] = playerdf["firstname"] + " " + playerdf["lastname"]

#This looks at the active scorers names and looks up the newly created FL column in the playerdf in order to get the top 20 active scorers player ids
scorerid = []
for player in currscorerslist:
    scorerid.append(playerdf.loc[playerdf['FL'] == player,"id"].iloc[0])

#creates the gamelog df
df = pd.DataFrame(columns=['id','firstname','lastname','pos','birthdate','daysalive','season','seasonid','gameid','assists','goals','sumgoals','pim','shots','games','hits','ppg','ppp','gwg','otg','shg','shp','bs','pm','pts','team','opp','date','home','win'])

#grabs the current season gamelogs for each top 20 active scorer
#create season paramaters list
seasonid = 20192020

for scorer in scorerid:

    try:
        #creates api call 
        person_url = f"http://statsapi.web.nhl.com/api/v1/people/{scorer}/"
        person = requests.get(person_url).json()

        #tests api to make sure it has data
        person["people"][0]["id"]

        try:
            #test if the player played in this season.  If not, it will produce an error and go to the next season
            gamelog_url = f"http://statsapi.web.nhl.com/api/v1/people/{scorer}/stats/?stats=gameLog&season={seasonid}"
            gamelogs = requests.get(gamelog_url).json()
            gamelogs["stats"][0]["splits"][0]

            #Loop through game logs within the season and grab various stats
            for game in range(len(gamelogs["stats"][0]["splits"])):

                #Player info
                df.loc[len(df.index), "id"] = scorer
                df.loc[len(df.index)-1, "firstname"] = person["people"][0]["firstName"]
                df.loc[len(df.index)-1, "lastname"] = person["people"][0]["lastName"]
                df.loc[len(df.index)-1, "birthdate"] = person["people"][0]["birthDate"]
                df.loc[len(df.index)-1, "pos"] = person["people"][0]['primaryPosition']['code'] 

                #Calculate days alive based on game date - birhdate
                origindate = datetime.strptime(df.loc[len(df.index)-1, "birthdate"], '%Y-%m-%d')
                lastdate = datetime.strptime(gamelogs["stats"][0]["splits"][game]["date"] , '%Y-%m-%d')
                df.loc[len(df.index)-1, "daysalive"] = (lastdate - origindate).days

                #adds season & season id
                df.loc[len(df.index)-1, "season"] = gamelogs["stats"][0]["splits"][0]["season"]
                df.loc[len(df.index)-1, "seasonid"] = seasonid

                #Stats pull...API address for stats: gamelogs["stats"][0]["splits"][0]["stat"]
                try:
                    df.loc[len(df.index)-1, "assists"] = gamelogs["stats"][0]["splits"][game]["stat"]["assists"]
                except (KeyError, IndexError):
                    pass#print("Missing assists field/result... skipping.")    
                try:
                    df.loc[len(df.index)-1, "goals"] = gamelogs["stats"][0]["splits"][game]["stat"]["goals"]
                except (KeyError, IndexError):
                    pass#print("Missing goals field/result... skipping.")    
                try:
                    df.loc[len(df.index)-1, "pim"] = gamelogs["stats"][0]["splits"][game]["stat"]["pim"]
                except (KeyError, IndexError):
                    pass#print("Missing pim field/result... skipping.")    
                try:
                    df.loc[len(df.index)-1, "shots"] = gamelogs["stats"][0]["splits"][game]["stat"]["shots"]
                except (KeyError, IndexError):
                    pass#print("Missing shots field/result... skipping.")    
                try:
                    df.loc[len(df.index)-1, "games"] = gamelogs["stats"][0]["splits"][game]["stat"]["games"]
                except (KeyError, IndexError):
                    pass#print("Missing games field/result... skipping.")    
                try:
                    df.loc[len(df.index)-1, "hits"] = gamelogs["stats"][0]["splits"][game]["stat"]["hits"]
                except (KeyError, IndexError):
                    pass#print("Missing hits field/result... skipping.")    
                try:
                    df.loc[len(df.index)-1, "ppg"] = gamelogs["stats"][0]["splits"][game]["stat"]["powerPlayGoals"]
                except (KeyError, IndexError):
                    pass#print("Missing ppg field/result... skipping.")    
                try:
                    df.loc[len(df.index)-1, "ppp"] = gamelogs["stats"][0]["splits"][game]["stat"]["powerPlayPoints"]
                except (KeyError, IndexError):
                    pass#print("Missing ppp field/result... skipping.")     
                try:
                    df.loc[len(df.index)-1, "gwg"] = gamelogs["stats"][0]["splits"][game]["stat"]["gameWinningGoals"]
                except (KeyError, IndexError):
                    pass#print("Missing gwg field/result... skipping.")    
                try:
                    df.loc[len(df.index)-1, "otg"] = gamelogs["stats"][0]["splits"][game]["stat"]["overTimeGoals"]
                except (KeyError, IndexError):
                    pass#print("Missing otg field/result... skipping.")    
                try:
                    df.loc[len(df.index)-1, "shg"] = gamelogs["stats"][0]["splits"][game]["stat"]["shortHandedGoals"]
                except (KeyError, IndexError):
                    pass#print("Missing shg field/result... skipping.")    
                try:
                    df.loc[len(df.index)-1, "shp"] = gamelogs["stats"][0]["splits"][game]["stat"]["shortHandedPoints"]
                except (KeyError, IndexError):
                    pass#print("Missing shp field/result... skipping.")    
                try:
                    df.loc[len(df.index)-1, "bs"] = gamelogs["stats"][0]["splits"][game]["stat"]["blocked"]
                except (KeyError, IndexError):
                    pass#print("Missing bs field/result... skipping.")    
                try:
                    df.loc[len(df.index)-1, "pm"] = gamelogs["stats"][0]["splits"][game]["stat"]["plusMinus"]
                except (KeyError, IndexError):
                    pass#print("Missing pm field/result... skipping.")    
                try:
                    df.loc[len(df.index)-1, "pts"] = gamelogs["stats"][0]["splits"][game]["stat"]["points"]
                except (KeyError, IndexError):
                    pass#print("Missing pts field/result... skipping.")    

                #Game info
                try:
                    df.loc[len(df.index)-1, "team"] = gamelogs["stats"][0]["splits"][game]["team"]["name"]
                except (KeyError, IndexError):
                    pass#print("Missing team field/result... skipping.")    
                try:
                    df.loc[len(df.index)-1, "opp"] = gamelogs["stats"][0]["splits"][game]["opponent"]["name"]
                except (KeyError, IndexError):
                    pass#print("Missing opp field/result... skipping.")    
                try:
                    df.loc[len(df.index)-1, "date"] = gamelogs["stats"][0]["splits"][game]["date"]
                except (KeyError, IndexError):
                    pass#print("Missing date field/result... skipping.")    
                try:
                    df.loc[len(df.index)-1, "home"] = gamelogs["stats"][0]["splits"][game]["isHome"]
                except (KeyError, IndexError):
                    pass#print("Missing home field/result... skipping.")    
                try:
                    df.loc[len(df.index)-1, "win"] = gamelogs["stats"][0]["splits"][game]["isWin"]
                except (KeyError, IndexError):
                    pass#print("Missing win field/result... skipping.")

        #error if the player didn't play in this season
        except (KeyError, IndexError):
            pass#print(f"Missing season {season}... skipping.") 

        print(f"-----Player ID: {scorer} complete-----")

    #error if player is missing   
    except (KeyError, IndexError):
            print(f"Missing playerid: {scorer}... skipping.")

#sort by days alive
df.sort_values(['id','daysalive'], inplace=True)
df.reset_index(drop=True, inplace=True)

df["id"] = df["id"].apply(pd.to_numeric)

####EVENTUALLY NEED TO FIGURE OUT HOW TO COMBINE HISTORICAL & CURRENT TABLES AND RUN THE FOLLOWING CODE

# # #calculate cumulative goals & game ID
# df['sumgoals'] = df.groupby('id')['goals'].transform(pd.Series.cumsum)
# df['gameid'] = df.groupby('id').cumcount() + 1



####EVENTUALLY NEED TO FIGURE OUT HOW TO COMBINE HISTORICAL & CURRENT TABLES AND RUN THE FOLLOWING CODE

# # #calculate cumulative goals & game ID
# df['sumgoals'] = df.groupby('id')['goals'].transform(pd.Series.cumsum)
# df['gameid'] = df.groupby('id').cumcount() + 1






-----Player ID: 8471214 complete-----
-----Player ID: 8466139 complete-----
-----Player ID: 8466148 complete-----
-----Player ID: 8471675 complete-----
-----Player ID: 8469454 complete-----
-----Player ID: 8470595 complete-----
-----Player ID: 8474564 complete-----
-----Player ID: 8466138 complete-----
-----Player ID: 8471215 complete-----
-----Player ID: 8468483 complete-----
-----Player ID: 8474141 complete-----
-----Player ID: 8470610 complete-----
-----Player ID: 8470604 complete-----
-----Player ID: 8470621 complete-----
-----Player ID: 8470598 complete-----
-----Player ID: 8473548 complete-----
-----Player ID: 8470794 complete-----
-----Player ID: 8470638 complete-----
-----Player ID: 8475166 complete-----
-----Player ID: 8473604 complete-----


In [3]:
conn = engine.connect()

df.to_sql(name='top20_active_GL_current_data', if_exists='replace', con=conn, chunksize=500, index=False)



In [4]:
df_hist = pd.read_sql('SELECT * FROM top20_active_GL_historic_data', con=engine)
df_hist.head()

,id,firstname,lastname,pos,birthdate,daysalive,season,seasonid,gameid,assists,...,shg,shp,bs,pm,pts,team,opp,date,home,win
0,8466138,Joe,Thornton,C,1979-07-02,6673,19971998,1,1,0,...,0,0,0,0,0,Boston Bruins,Phoenix Coyotes,1997-10-08,0,1
1,8466138,Joe,Thornton,C,1979-07-02,6676,19971998,1,2,0,...,0,0,0,-1,0,Boston Bruins,San Jose Sharks,1997-10-11,0,0
2,8466138,Joe,Thornton,C,1979-07-02,6678,19971998,1,3,0,...,0,0,0,0,0,Boston Bruins,Anaheim Ducks,1997-10-13,0,1
3,8466138,Joe,Thornton,C,1979-07-02,6680,19971998,1,4,0,...,0,0,0,0,0,Boston Bruins,Los Angeles Kings,1997-10-15,0,1
4,8466138,Joe,Thornton,C,1979-07-02,6682,19971998,1,5,0,...,0,0,0,0,0,Boston Bruins,Vancouver Canucks,1997-10-17,0,1


In [5]:
# df_curr = pd.read_sql('SELECT * FROM top20_active_GL_current_data', con=engine)
# df_curr.head()

,id,firstname,lastname,pos,birthdate,daysalive,season,seasonid,gameid,assists,...,shg,shp,bs,pm,pts,team,opp,date,home,win
0,8466138,Joe,Thornton,C,1979-07-02,14702,20192020,20192020,None,0,...,0,0,1,-2,0,San Jose Sharks,Vegas Golden Knights,2019-10-02,0,0
1,8466138,Joe,Thornton,C,1979-07-02,14704,20192020,20192020,None,1,...,0,0,0,-2,1,San Jose Sharks,Vegas Golden Knights,2019-10-04,1,0
2,8466138,Joe,Thornton,C,1979-07-02,14705,20192020,20192020,None,0,...,0,0,0,0,0,San Jose Sharks,Anaheim Ducks,2019-10-05,0,0
3,8466138,Joe,Thornton,C,1979-07-02,14708,20192020,20192020,None,0,...,0,0,0,0,0,San Jose Sharks,Nashville Predators,2019-10-08,0,0
4,8466138,Joe,Thornton,C,1979-07-02,14710,20192020,20192020,None,1,...,0,0,0,-1,1,San Jose Sharks,Chicago Blackhawks,2019-10-10,0,1


In [9]:
df_combine = pd.concat([df, df_hist])
df_combine

,id,firstname,lastname,pos,birthdate,daysalive,season,seasonid,gameid,assists,...,shg,shp,bs,pm,pts,team,opp,date,home,win
0,8466138,Joe,Thornton,C,1979-07-02,14702,20192020,20192020,NaN,0,...,0,0,1,-2,0,San Jose Sharks,Vegas Golden Knights,2019-10-02,False,False
1,8466138,Joe,Thornton,C,1979-07-02,14704,20192020,20192020,NaN,1,...,0,0,0,-2,1,San Jose Sharks,Vegas Golden Knights,2019-10-04,True,False
2,8466138,Joe,Thornton,C,1979-07-02,14705,20192020,20192020,NaN,0,...,0,0,0,0,0,San Jose Sharks,Anaheim Ducks,2019-10-05,False,False
3,8466138,Joe,Thornton,C,1979-07-02,14708,20192020,20192020,NaN,0,...,0,0,0,0,0,San Jose Sharks,Nashville Predators,2019-10-08,False,False
4,8466138,Joe,Thornton,C,1979-07-02,14710,20192020,20192020,NaN,1,...,0,0,0,-1,1,San Jose Sharks,Chicago Blackhawks,2019-10-10,False,True
5,8466138,Joe,Thornton,C,1979-07-02,14713,20192020,20192020,NaN,0,...,0,0,0,0,0,San Jose Sharks,Calgary Flames,2019-10-13,True,True
6,8466138,Joe,Thornton,C,1979-07-02,14716,20192020,20192020,NaN,0,...,0,0,0,0,0,San Jose Sharks,Carolina Hurricanes,2019-10-16,True,True
7,8466138,Joe,Thornton,C,1979-07-02,14719,20192020,20192020,NaN,1,...,0,0,0,-1,1,San Jose Sharks,Buffalo Sabres,2019-10-19,True,False
8,8466138,Joe,Thornton,C,1979-07-02,14722,20192020,20192020,NaN,0,...,0,0,0,0,0,San Jose Sharks,Buffalo Sabres,2019-10-22,False,False
9,8466138,Joe,Thornton,C,1979-07-02,14724,20192020,20192020,NaN,1,...,0,0,1,1,1,San Jose Sharks,Montréal Canadiens,2019-10-24,False,True


In [10]:
#sort by days alive
df_combine.sort_values(['id','daysalive'], inplace=True)
df_combine.reset_index(drop=True, inplace=True)

###EVENTUALLY NEED TO FIGURE OUT HOW TO COMBINE HISTORICAL & CURRENT TABLES AND RUN THE FOLLOWING CODE

# #calculate cumulative goals & game ID
df_combine['sumgoals'] = df_combine.groupby('id')['goals'].transform(pd.Series.cumsum)
df_combine['gameid'] = df_combine.groupby('id').cumcount() + 1
df_combine.head()

,id,firstname,lastname,pos,birthdate,daysalive,season,seasonid,gameid,assists,...,shg,shp,bs,pm,pts,team,opp,date,home,win
0,8466138,Joe,Thornton,C,1979-07-02,6673,19971998,1,1,0,...,0,0,0,0,0,Boston Bruins,Phoenix Coyotes,1997-10-08,0,1
1,8466138,Joe,Thornton,C,1979-07-02,6676,19971998,1,2,0,...,0,0,0,-1,0,Boston Bruins,San Jose Sharks,1997-10-11,0,0
2,8466138,Joe,Thornton,C,1979-07-02,6678,19971998,1,3,0,...,0,0,0,0,0,Boston Bruins,Anaheim Ducks,1997-10-13,0,1
3,8466138,Joe,Thornton,C,1979-07-02,6680,19971998,1,4,0,...,0,0,0,0,0,Boston Bruins,Los Angeles Kings,1997-10-15,0,1
4,8466138,Joe,Thornton,C,1979-07-02,6682,19971998,1,5,0,...,0,0,0,0,0,Boston Bruins,Vancouver Canucks,1997-10-17,0,1
